In [1]:
import openai, pandas as pd, requests, bs4, readability, lxml, praw, os
print("OK")

OK


In [15]:
from dotenv import load_dotenv; load_dotenv()
import os
print("OPENAI:", bool(os.getenv("OPENAI_API_KEY")))
print("SERPAPI:", bool(os.getenv("SERPAPI_API_KEY")))
print("REDDIT:", all([os.getenv("REDDIT_CLIENT_ID"), os.getenv("REDDIT_CLIENT_SECRET"), os.getenv("REDDIT_USER_AGENT")]))


OPENAI: True
SERPAPI: True
REDDIT: True


In [ ]:
# --- Stage 0: Planner — propose subreddits & queries from a prior markdown report ---

import os, re, json, pathlib
from openai import OpenAI
from dotenv import load_dotenv; load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

INSIGHTS_FILENAME = "insights_20251025_1652.md"   # <- change per run as needed
INSIGHTS_PATH = pathlib.Path("../data/exports") / INSIGHTS_FILENAME

assert INSIGHTS_PATH.exists(), f"Missing file: {INSIGHTS_PATH}"
ctx_md = INSIGHTS_PATH.read_text(encoding="utf-8")

# ----- Structured outputs schema: object with arrays -----
RESULT_SCHEMA = {
    "type": "object",
    "additionalProperties": False,
    "properties": {
        "subreddits": {
            "type": "array",
            "items": {"type": "string", "minLength": 3, "maxLength": 32},
            "maxItems": 10
        },
        "queries": {
            "type": "array",
            "items": {"type": "string", "minLength": 3, "maxLength": 200},
            "maxItems": 10
        },
        # optional notes/rationale (not used downstream but handy for debugging)
        "notes": {"type": ["string", "null"]}
    },
    "required": ["subreddits", "queries"]
}

SYSTEM = (
    "You are a senior UX researcher and search strategist. "
    "Given a research summary + instructions, output up to 10 relevant Reddit subreddits "
    "and up to 10 focused search queries to discover pain points for private-party used-car transactions."
)

USER = f"""
CONTEXT (Markdown from prior run):
---
{ctx_md}
---

TASK:
- Generate up to 10 subreddits that are most likely to contain firsthand experiences, how-tos, and pitfalls relevant to private-party used-car buying/selling in the U.S.
- Generate up to 10 search queries tuned for Reddit search, covering payments/scams, title/lien, inspections/OBD, negotiating, financing, insurance, paperwork, scheduling/meeting, post-purchase issues.
- Prefer **practical, specific** queries that surface stories (e.g., "private party title transfer lien release", "cashier's check scam private sale", "mobile mechanic OBD pre-purchase").
- Exclude dealership-only communities or irrelevant car culture subs unless they have frequent private-sale threads.

OUTPUT:
Return a JSON object with:
- "subreddits": string[]
- "queries": string[]
- "notes": optional rationale

Guidelines:
- Do NOT include the 'r/' prefix.
- Avoid duplicates and overly-broad terms like just "used car".
- Favor U.S. context unless the summary indicates otherwise.
"""

chat = client.chat.completions.create(
    model="gpt-4.1-mini",
    temperature=0.2,
    messages=[{"role":"system","content": SYSTEM},
              {"role":"user","content": USER}],
    response_format={
        "type": "json_schema",
        "json_schema": {"name":"PlanSpec","schema":RESULT_SCHEMA,"strict":True}
    }
)

import json as _json
plan = _json.loads(chat.choices[0].message.content)

# ----- Normalize / validate -----
def norm_sub(s: str) -> str:
    s = s.strip()
    s = re.sub(r"^(?:/)?r/", "", s, flags=re.I)  # drop r/ or /r/
    s = re.sub(r"[^A-Za-z0-9_]", "", s)          # keep reddit-safe chars
    return s

SUGGESTED_SUBREDDITS = []
seen = set()
for s in plan.get("subreddits", []):
    n = norm_sub(s)
    if 3 <= len(n) <= 32 and n.lower() not in seen:
        SUGGESTED_SUBREDDITS.append(n)
        seen.add(n.lower())

# trim queries, dedupe
SUGGESTED_QUERIES = []
seenq = set()
for q in plan.get("queries", []):
    qn = " ".join(q.split())
    if 3 <= len(qn) <= 200 and qn.lower() not in seenq:
        SUGGESTED_QUERIES.append(qn)
        seenq.add(qn.lower())

print("Planner → suggested subreddits:", SUGGESTED_SUBREDDITS)
print("Planner → suggested queries:", SUGGESTED_QUERIES)
if plan.get("notes"):
    print("\nNotes:\n", plan["notes"])


In [16]:
# Configuation

# --- config ---
TOPIC = "Private-party used-car buying in SF Bay Area"
SUBREDDITS = ["UsedCars", "MechanicAdvice", "WhatCarShouldIBuy", "cars", "Scams", "AskCarsales"]
QUERY_STRINGS = [
    'Buying a used car',
    'private party payment cashier check',
    'private party title lien transfer',
    'private party escrow payment',
    'inspection OBD checklist private sale',
    'wire vs cash private sale',
]


RECENCY_MONTHS = 24
SINCE_DAYS = RECENCY_MONTHS * 30  # Stage 1 uses days
THREADS_PER_QUERY = 4   # keep small for first run
TOP_COMMENTS = 4

import os, json, time, math, datetime as dt, re, pathlib # re = pyton regex. pathlib = an easy way to manage env vars.
import praw as praw # Reddit API
from openai import OpenAI # OpenAI API

# loads the environment vars from .env
from dotenv import load_dotenv; load_dotenv()

# Creates an env var for the outputs, and creates a data/exports dir that corresponds with the env var
EXPORTS = pathlib.Path("../data/exports"); EXPORTS.mkdir(parents=True, exist_ok=True)


RUN_ID = dt.datetime.utcnow().strftime("%Y%m%d_%H%M")
print("Run:", RUN_ID)


Run: 20251025_1652


In [ ]:
# Configuration - new

# --- topic ---
TOPIC = "Private-party used-car buying in the US"

# --- use suggestions from Stage 0 if present; else fall back ---
DEFAULT_SUBS = ["UsedCars", "MechanicAdvice", "WhatCarShouldIBuy", "cars", "Scams", "AskCarsales"]
DEFAULT_QUERIES = [
    "Buying a used car",
    "private party payment cashier check",
    "private party title lien transfer",
    "private party escrow payment",
    "inspection OBD checklist private sale",
    "wire vs cash private sale",
]

SUBREDDITS = SUGGESTED_SUBREDDITS if 'SUGGESTED_SUBREDDITS' in globals() and SUGGESTED_SUBREDDITS else DEFAULT_SUBS
QUERY_STRINGS = SUGGESTED_QUERIES if 'SUGGESTED_QUERIES' in globals() and SUGGESTED_QUERIES else DEFAULT_QUERIES

# knobs
RECENCY_MONTHS = 24
SINCE_DAYS = RECENCY_MONTHS * 30
THREADS_PER_QUERY = 4
TOP_COMMENTS = 4

import os, json, time, math, datetime as dt, re, pathlib # re = pyton regex. pathlib = an easy way to manage env vars.
import praw as praw # Reddit API
from openai import OpenAI # OpenAI API
from dotenv import load_dotenv; load_dotenv() # loads the environment vars from .env


# Creates an env var for the outputs, and creates a data/exports dir that corresponds with the env var
EXPORTS = pathlib.Path("../data/exports"); EXPORTS.mkdir(parents=True, exist_ok=True)
RUN_ID = dt.datetime.utcnow().strftime("%Y%m%d_%H%M")
print("Run:", RUN_ID)
print("Using subreddits:", SUBREDDITS)
print("Using queries:", QUERY_STRINGS)


In [17]:
# --- Stage 1: Researcher (Reddit via PRAW search) ---

# connect to Reddit (uses your .env values)
reddit = praw.Reddit(
    client_id=os.getenv("REDDIT_CLIENT_ID"),
    client_secret=os.getenv("REDDIT_CLIENT_SECRET"),
    user_agent=os.getenv("REDDIT_USER_AGENT"),
)

reddit.read_only = True

# simple call: fetch one hot post from r/UsedCars
#sub = reddit.subreddit("UsedCars")
#post = next(sub.hot(limit=1))
#print("OK:", post.title[:80])


def search_threads(query, subreddits, limit_per_sub, since_days):
    """ Search Reddit for `query` across `subreddits`, keeping posts newer than `since_days`.

    Args:
        query: Search string (e.g., "private party escrow").
        subreddits: List like ["UsedCars", "MechanicAdvice"].
        limit_per_sub: Max results per subreddit (before filtering).
        since_days: Recency window; older posts are dropped.

    Returns:
        A de-duplicated list of thread dicts with id, title, permalink, timestamps, etc.
    """
    
    after_ts = time.time() - since_days * 24 * 3600
    found = []
    
    for sub in subreddits:
        sr = reddit.subreddit(sub)
        # Reddit search (relevance) with a per-subreddit cap
        for submission in sr.search(query, sort="relevance", limit=limit_per_sub):
            if submission.created_utc < after_ts:
                continue
            found.append({
                "id": submission.id,
                "subreddit": str(sub),
                "title": submission.title,
                "permalink": "https://www.reddit.com" + submission.permalink,
                "created_utc": submission.created_utc,
                "created_iso": dt.datetime.utcfromtimestamp(submission.created_utc).isoformat() + "Z",
                "is_self": submission.is_self,
                "url": submission.url,
                "query": query,
            })
    # de-dupe by submission id
    seen = set()
    dedup = []
    for r in found:
        if r["id"] in seen:
            continue
        seen.add(r["id"])
        dedup.append(r)
    return dedup


# run searches over your configured queries/subreddits
all_threads = []
for q in QUERY_STRINGS:
    all_threads += search_threads(
        query=q, 
        subreddits=SUBREDDITS, 
        limit_per_sub=THREADS_PER_QUERY, 
        since_days=SINCE_DAYS
    )

# global de-dup (across queries)
seen_ids, threads = set(), []
for r in all_threads:
    if r["id"] in seen_ids: 
        continue
    seen_ids.add(r["id"]); threads.append(r)

print(f"Threads found (after recency + dedupe): {len(threads)}")
# quick peek
for t in threads[:5]:
    print(f"- r/{t['subreddit']} | {t['created_iso']} | {t['title'][:200]}…") 



Threads found (after recency + dedupe): 80
- r/UsedCars | 2025-10-11T17:56:53Z | how do you buy a used car?…
- r/UsedCars | 2025-08-01T20:49:40Z | Is buying a used car still the way to go?…
- r/UsedCars | 2025-08-07T16:51:57Z | How do people even buy used cars.…
- r/UsedCars | 2025-07-25T15:23:52Z | I sold a classic car to this guy (private sale) and now he's saying I misrepresented the car and he's giving me the opportunity to buy the car back.…
- r/MechanicAdvice | 2025-07-17T05:27:12Z | Is there any used car worth buying with high miles?…


In [18]:
# --- Stage 2: Crawler — expand each thread with post + top comments ---

def fetch_thread(submission_id: str, top_n: int = TOP_COMMENTS) -> dict:
    sub = reddit.submission(id=submission_id)
    sub.comment_sort = "top"
    # flatten "More comments…" so we get real comments
    sub.comments.replace_more(limit=0)

    comments = []
    for c in sub.comments[:top_n]:
        comments.append({
            "author": str(c.author) if c.author else "deleted",
            "body": c.body,
            "permalink": f"https://www.reddit.com{c.permalink}",
            "created_utc": c.created_utc,
        })

    return {
        "thread_id": sub.id,
        "subreddit": str(sub.subreddit),
        "thread_title": sub.title,
        "selftext": sub.selftext or "",
        "permalink": f"https://www.reddit.com{sub.permalink}",
        "created_utc": sub.created_utc,
        "comments": comments,
    }

# run crawler over the threads we just found
bundles = []
for t in threads:
    try:
        bundles.append(fetch_thread(t["id"], top_n=TOP_COMMENTS))
    except Exception as e:
        print("crawl error:", t["id"], e)


def preview_bundles(bundles, max_title=100, max_comment=140):
    for idx, b in enumerate(bundles, 1):
        print(f"\n[{idx}] r/{b['subreddit']} | {b['thread_title'][:max_title]}")
        print(f"    Thread: {b['permalink']}")
        for i, c in enumerate(b["comments"], 1):
            body = " ".join(c["body"].split())  # collapse whitespace/newlines
            excerpt = (body[:max_comment] + "…") if len(body) > max_comment else body
            print(f"    {i:02d}. {excerpt}")
        print("-"*80)

#preview_bundles(bundles)

# Print into a json file
out_path = EXPORTS / f"bundles_{RUN_ID}.json"
with open(out_path, "w") as f:
    json.dump(bundles, f, ensure_ascii=False, indent=2)
str(out_path)



'../data/exports/bundles_20251025_1652.json'

In [19]:
# --- Stage 3.1 Pre-LLM dedupe of bundles (saves tokens) ---

import hashlib, re

def sig_for_bundle(b):
    """Signature of thread content for dedupe: title + selftext + top comment bodies (normalized)."""
    text_parts = [b.get("thread_title",""), b.get("selftext","")]
    text_parts += [c.get("body","") for c in b.get("comments", [])]
    raw = " ".join(text_parts).lower()
    raw = re.sub(r"\s+", " ", raw).strip()
    return hashlib.md5(raw.encode("utf-8")).hexdigest()

seen_sig = set()
bundles_dedup = []
for b in bundles:
    sig = sig_for_bundle(b)
    if sig in seen_sig: 
        continue
    seen_sig.add(sig)
    bundles_dedup.append(b)

print(f"Bundles before LLM: {len(bundles)} → after pre-dedupe: {len(bundles_dedup)}")


Bundles before LLM: 80 → after pre-dedupe: 80


In [20]:
# --- Stage 3: Extractor — LLM → structured rows ---
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

SCHEMA = {
 "persona": "buyer|seller|unknown",
 "stage": "initial research | choosing the right car | inspection and car condition | scheduling and meeting the other party | negotiation and pricing | payment | paperwork | financing | insurance | post purchase",
 "action": "what they did",
 "feeling": "how they felt",
 "pain": "risk/annoyance/issue (null if none)",
 "workaround": "what they tried (null if none)",
 "opportunity": "What value we could create to help (null if none)",
 "verbatim_quote": "exact sentence from source",
 "source_url": "https://...",
 "permalink": "https://...#comment-id (or thread permalink)",
 "source_type": "reddit",
 "thread_title": "...",
 "posted_at": "ISO8601 or null",
 "location_city": "city name or null",
 "location_state": "US state/region or null",
 "location_hint": "free text if mentioned (dealer/DMV names, banks, neighborhoods) or null",
 "potential_competitor": "name/url of service used or mentioned as a solution, else null",
 "stage_confidence": 0.0,
 "persona_confidence": 0.0
}


EXTRACT_GUIDE = """
Act as a UX researcher analyzing PRIVATE-PARTY used-car transactions (person buys from a private seller, not a dealer).
You will read SOURCE TEXT (thread title/body and top comments) and output ONLY a JSON array of objects following the SCHEMA exactly.

GOAL
- Detect concrete PAIN POINTS expressed or clearly implied in the text.
- For each distinct pain point (or strong observation with clear action/feeling), emit ONE object.
- If there is no concrete pain/observation, return [].

STRICT RULES
1) JSON ONLY. No prose, no markdown, no trailing commas.
2) Use the provided enum values EXACTLY:
   - persona: buyer|seller|unknown
   - stage: one of
     "initial research" | "choosing the right car" | "inspection and car condition" |
     "scheduling and meeting the other party" | "negotiation and pricing" | "payment" |
     "paperwork" | "financing" | "insurance" | "post purchase"
3) Evidence:
   - Include at least ONE verbatim sentence from the SOURCE TEXT as "verbatim_quote".
     • Do not paraphrase in that field.
   - Use the comment's permalink if quoting a comment; otherwise the thread permalink.
4) Fields:
   - Keep "action", "feeling", "pain", "workaround", "opportunity" concise and pragmatic.
   - If a field isn’t present in the text, set it to null (do NOT invent).
   - Extract location if present:
     • location_city and location_state when clearly mentioned (e.g., "San Jose, CA"), else null
     • location_hint for looser clues (DMV names, banks, neighborhoods), else null
   - potential_competitor: name and/or URL of any service/app used/mentioned to solve the issue (escrow, inspection, payments, insurance, financing), else null
5) Confidence:
   - Set stage_confidence and persona_confidence between 0.0 and 1.0 based on textual clarity.
6) Scope control:
   - Only include content about private-party used-car buying/selling. Ignore dealership stories unless directly relevant to a private-party comparison.

OUTPUT FORMAT
- Return a JSON object with a single key "rows".
- "rows" must be a JSON array of objects matching the schema.
- If there are no qualifying observations, return {"rows": []}.
"""



# --- Structured Outputs schema (object with rows array) ---
ROW_SCHEMA = {
    "type": "object",
    "additionalProperties": False,
    "properties": {
        "persona": {"type":"string","enum":["buyer","seller","unknown"]},
        "stage": {"type":"string","enum":[
            "initial research","choosing the right car","inspection and car condition",
            "scheduling and meeting the other party","negotiation and pricing",
            "payment","paperwork","financing","insurance","post purchase"
        ]},
        "action": {"type":["string","null"]},
        "feeling": {"type":["string","null"]},
        "pain": {"type":["string","null"]},
        "workaround": {"type":["string","null"]},
        "opportunity": {"type":["string","null"]},
        "verbatim_quote": {"type":"string"},
        "source_url": {"type":"string"},
        "permalink": {"type":"string"},
        "source_type": {"type":"string","const":"reddit"},
        "thread_title": {"type":"string"},
        "posted_at": {"type":["string","null"]},
        "location_city": {"type":["string","null"]},
        "location_state": {"type":["string","null"]},
        "location_hint": {"type":["string","null"]},
        "potential_competitor": {"type":["string","null"]},
        "stage_confidence": {"type":"number"},
        "persona_confidence": {"type":"number"}
    },
    "required": [
        "persona","stage","action","feeling","pain","workaround","opportunity",
        "verbatim_quote","source_url","permalink","source_type","thread_title",
        "posted_at","location_city","location_state","location_hint",
        "potential_competitor","stage_confidence","persona_confidence"
    ]
}

RESULT_SCHEMA = {
    "type": "object",
    "additionalProperties": False,
    "properties": {
        "rows": {
            "type": "array",
            "items": ROW_SCHEMA
        }
    },
    "required": ["rows"]
}

#ARRAY_SCHEMA = {"type":"array","items": ROW_SCHEMA}


def extract_from_bundle(b):
    parts = [f"THREAD: {b['thread_title']}", b.get("selftext","")]
    for c in b.get("comments", []):
        parts.append(f"COMMENT ({c['permalink']}): {c['body']}")
    text = "\n\n".join(parts)[:20000]

    chat = client.chat.completions.create(
        model="gpt-4.1-mini",  # or "o3-mini"
        temperature=0.15,
        messages=[
            {"role":"system","content":"You extract structured observations for private-party used-car journeys."},
            {"role":"user","content": json.dumps({
                "schema": SCHEMA,
                "thread_permalink": b["permalink"],
                "text": text
            })},
            {"role":"user","content": EXTRACT_GUIDE}
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {"name":"RowObject","schema": RESULT_SCHEMA, "strict": True}
        }
    )

    import json as _json
    try:
        obj = _json.loads(chat.choices[0].message.content)  # object with "rows"
        arr = obj.get("rows", [])
    except Exception as e:
        print("parse error:", e)
        return []

    # bound per-thread, optional
    MAX_ROWS_PER_THREAD = 6
    arr = arr[:MAX_ROWS_PER_THREAD]

    # normalize/stamp
    for r in arr:
        r.setdefault("source_url", b["permalink"])
        r.setdefault("thread_title", b["thread_title"])
        r.setdefault("source_type", "reddit")
        if not r.get("permalink"): r["permalink"] = b["permalink"]
        for k in ("stage_confidence","persona_confidence"):
            try:
                v = float(r.get(k, 0.0)); r[k] = max(0.0, min(1.0, v))
            except Exception:
                r[k] = 0.0
    return arr



In [21]:
# Stage 3.3
from tqdm.auto import tqdm
import json

RAW_DIR = EXPORTS / f"raw_{RUN_ID}"
RAW_DIR.mkdir(parents=True, exist_ok=True)

def generate_and_cache(b, idx):
    target = RAW_DIR / f"{idx:04d}.json"
    if target.exists():
        return
    arr = extract_from_bundle(b)          # LLM call ONCE
    payload = {"thread_permalink": b["permalink"], "thread_title": b["thread_title"], "rows": arr}
    with open(target, "w") as f:
        json.dump(payload, f, ensure_ascii=False, indent=2)

for i, b in enumerate(tqdm(bundles_dedup, desc="LLM extract & cache"), start=1):
    generate_and_cache(b, i)

print("Cached to:", RAW_DIR)


LLM extract & cache:   0%|          | 0/80 [00:00<?, ?it/s]

Cached to: ../data/exports/raw_20251025_1652


In [25]:
# Stage 3B - Load cached rows
import json, glob

rows = []
files = sorted(glob.glob(str(RAW_DIR / "*.json")))
for fp in files:
    with open(fp, "r") as f:
        data = json.load(f)
        rows += data.get("rows", [])

print("Loaded rows from cache:", len(rows))


Loaded rows from cache: 160


In [26]:
# --- Stage 4: Normalize / Dedupe / Filter ---

# It drops query strings (?utm=…) and fragments (#comment-123) so two URLs that are the same page compare equal.
# Examples:
# https://site.com/page?utm_source=news#section → https://site.com/page
# https://reddit.com/r/UsedCars/comments/abc123/?sort=top → https://reddit.com/r/UsedCars/comments/abc123/
def canon_url(u: str) -> str:
    if not u: return ""
        
    # substitute ? or # and everything that comes after it with  "". Strip removes any leading or trailing spaces from a string
    return re.sub(r"[?#].*$", "", u.strip()) 

# Input
# r: dict — one extracted row from the LLM (a Python dict). It should have numeric fields:
#     stage_confidence
#     persona_confidence
#
# min_conf: float = 0.5 — the threshold both confidences must meet or exceed (defaults to 0.5).
#
# Output
# bool — True if both confidences are ≥ min_conf; otherwise False.
#
# Algorithm (one-liner, expanded)
# Look up stage_confidence and persona_confidence in r with .get(...).
# If a key is missing, use 0 as a default.
# Cast each value to float (handles strings like "0.72" or None safely—non-numeric strings would raise).
# Compare both to min_conf.
# Return True only if both comparisons pass (logical AND).
def ok_conf(r: dict, min_conf: float = 0.5) -> bool:
    return (
        float(r.get("stage_confidence", 0)) >= min_conf and
        float(r.get("persona_confidence", 0)) >= min_conf
    )

# Purpose
# Gate rows by recency: keep items whose timestamp is within the last months (default RECENCY_MONTHS). It’s a soft filter—if there’s no usable date, 
# it keeps the row rather than throwing it away.
#
# Signature (inputs → output)
# Input
# iso: str — a timestamp string in (roughly) ISO-8601 (e.g., "2024-07-15T12:34:56Z" or "2024-07-15T12:34:56").
# months: int = 24 — how many months back to allow (approximate, using 30 days per month).
#
# Output
# bool — True if the row should be kept, False if it’s too old.
def within_months(iso: str, months: int = RECENCY_MONTHS) -> bool:
    if not iso:
        return True
    try:
        d = dt.datetime.fromisoformat(iso.replace("Z",""))
        cutoff = dt.datetime.utcnow() - dt.timedelta(days=months*30)
        return d >= cutoff
    except Exception:
        return True

# Python list comprehensions
# [new_item   for item in iterable   if condition]
# Read it left→right: “Put new_item into a list for each item in iterable if condition.”

# Breakdown:
# r for r in rows → iterate all extracted row dicts.
# r.get("verbatim_quote") → fetches the value (or None if missing). In an if, this is truthy only if it exists and isn’t empty (e.g., not "").
# and r.get("source_url") → also require a non-empty source_url.
# The comprehension builds a new list with only rows that satisfy both conditions.
#
# Why:
# We require a real, verbatim quote (evidence) and a URL to verify it. Rows missing either are dropped before dedupe/QA.

# 1) must-haves
rows_clean = [r for r in rows if r.get("verbatim_quote") and r.get("source_url")]

# What it does
# Goal: keep only the first occurrence of a row with the same quote + URL.
# seen (set): fast membership check of keys we’ve already kept.
# key: a tuple of
# the quote text (trimmed), and
# a canonicalized URL (permalink if present, else source_url, with query/fragment stripped by canon_url).
# If the key was seen before → skip; otherwise record it and append the row to deduped.

# 2) de-dupe by (verbatim_quote, permalink-or-source_url)
seen = set(); deduped = []
for r in rows_clean:
    key = (r["verbatim_quote"].strip(), canon_url(r.get("permalink") or r.get("source_url")))
    if key in seen: 
        continue
    seen.add(key); deduped.append(r)

# 3) confidence gate
filtered = [r for r in deduped if ok_conf(r, min_conf=0.5)]

# 4) optional recency filter if posted_at present
filtered = [r for r in filtered if within_months(r.get("posted_at",""), months=24)]

print(f"Rows → raw: {len(rows)} | clean: {len(rows_clean)} | deduped: {len(deduped)} | final: {len(filtered)}")


Rows → raw: 160 | clean: 160 | deduped: 160 | final: 160


In [27]:
# --- Stage 5: Review (summary + preview) ---

import pandas as pd
from IPython.display import display

pd.set_option("display.max_colwidth", 160)

df = pd.DataFrame(filtered)

# 1) High-level summary: counts by persona / stage / source
summary = (
    df.groupby(["persona", "stage", "source_type"], dropna=False)
      .size()
      .reset_index(name="count")
      .sort_values(["persona","stage","count"], ascending=[True, True, False])
)
display(summary.head(30))

# 2) Distribution by stage (quick sanity)
stage_counts = df["stage"].value_counts().rename_axis("stage").reset_index(name="count")
display(stage_counts)

# 3) Human-friendly preview of the first N rows
preview_cols = [
    "persona","stage","verbatim_quote","opportunity",
    "permalink","thread_title","location_city","location_state"
]
preview = df[preview_cols].head(12)
display(preview)


# 4) (Optional) mark-and-drop any rows by index before saving
#    After you scan `preview` or `df`, put indices to drop here:
drop_idx = []   # e.g., [3, 7]
if drop_idx:
    df = df.drop(index=drop_idx).reset_index(drop=True)
    print(f"Dropped {len(drop_idx)} rows; remaining: {len(df)}")
else:
    print(f"No manual drops; rows: {len(df)}")

# Keep `df` as the reviewed set for Stage 6 save.
reviewed_rows = df.to_dict(orient="records")


,persona,stage,source_type,count
0,buyer,choosing the right car,reddit,6
1,buyer,financing,reddit,9
2,buyer,initial research,reddit,14
3,buyer,inspection and car condition,reddit,30
4,buyer,insurance,reddit,2
5,buyer,negotiation and pricing,reddit,7
6,buyer,paperwork,reddit,19
7,buyer,payment,reddit,19
8,buyer,post purchase,reddit,16
9,buyer,scheduling and meeting the other party,reddit,5


,stage,count
0,payment,35
1,inspection and car condition,30
2,paperwork,23
3,post purchase,19
4,initial research,14
5,negotiation and pricing,12
6,scheduling and meeting the other party,10
7,financing,9
8,choosing the right car,6
9,insurance,2


,persona,stage,verbatim_quote,opportunity,permalink,thread_title,location_city,location_state
0,buyer,initial research,"Well, if youve never bought one or been to a dealership, do not ever go in person. They see you coming a mile away and get you to buy something you dont ...",Provide trustworthy third party tools to handle dealer interactions and avoid pressure tactics,https://www.reddit.com/r/UsedCars/comments/1o426it/how_do_you_buy_a_used_car/nj4df06/,how do you buy a used car?,None,None
1,buyer,choosing the right car,"Vehicle history is important. Was it a rental? How many owners, any accidents, and how was the maintenance? Most dealerships will offer a car fax history re...",Offer comprehensive vehicle history and condition reports; facilitate trusted inspections,https://www.reddit.com/r/UsedCars/comments/1o426it/how_do_you_buy_a_used_car/nizb6pt/,how do you buy a used car?,None,None
2,buyer,inspection and car condition,"If you made it this far and want to take the extra step, especially if you didnt understand all of that. Drive it to a reputable shop across town. Pay for a...",Provide easy access to trusted inspection services and clear condition summaries,https://www.reddit.com/r/UsedCars/comments/1o426it/how_do_you_buy_a_used_car/nizb6pt/,how do you buy a used car?,None,None
3,buyer,initial research,"find the wealthiest zip code near you. \n\nrestrict your searches to that. \n\nwealthy folks take great care of their cars, and when they sell them private ...",Provide data-driven neighborhood quality indicators for used car searches,https://www.reddit.com/r/UsedCars/comments/1o426it/how_do_you_buy_a_used_car/nj0hkds/,how do you buy a used car?,None,None
4,buyer,financing,Always get your own financing. Get pre-approved letter from your bank. Dont use dealer financing. Dealer will lie and give you higher intrest rate.,Provide transparent financing options and pre-approval tools,https://www.reddit.com/r/UsedCars/comments/1o426it/how_do_you_buy_a_used_car/nizbjeb/,how do you buy a used car?,None,None
5,buyer,inspection and car condition,Look at oilchange sticker on window. Is it overdue for oil change? If its overdue than a major red flag.,Provide maintenance history indicators to buyers,https://www.reddit.com/r/UsedCars/comments/1o426it/how_do_you_buy_a_used_car/nizbjeb/,how do you buy a used car?,None,None
6,buyer,initial research,"Then Covid happened and used car prices soared. I expect that has mostly corrected, but I wonder.",provide updated market pricing info to help buyers assess value,https://www.reddit.com/r/UsedCars/comments/1mf7v5k/is_buying_a_used_car_still_the_way_to_go/n6f5k2z/,Is buying a used car still the way to go?,None,None
7,buyer,choosing the right car,"The downside to this method is leg work. Also, finding someone to inspect the car for you.",offer inspection services or trusted inspectors to reduce buyer effort and risk,https://www.reddit.com/r/UsedCars/comments/1mf7v5k/is_buying_a_used_car_still_the_way_to_go/n6fkfk7/,Is buying a used car still the way to go?,None,None
8,buyer,post purchase,"QC is worse than it ever has been in my lifetime with vehicles and if these new, complex engines arent properly maintained they just don't last. Maybe I'm j...",provide reliable maintenance history and quality assurance for used cars,https://www.reddit.com/r/UsedCars/comments/1mf7v5k/is_buying_a_used_car_still_the_way_to_go/n6f23mu/,Is buying a used car still the way to go?,None,None
9,buyer,scheduling and meeting the other party,"They either don't reply, or they do but ghost me after a few messages, or they set up a meeting time then message to say it's sold.",help buyers connect reliably with sellers and reduce ghosting,https://www.reddit.com/r/UsedCars/comments/1mk5cyg/how_do_people_even_buy_used_cars/,How do people even buy used cars.,None,None


No manual drops; rows: 160


In [29]:
# --- Stage 6: Insight Summarization (UX-research style) ---

import os, json, random, pandas as pd, datetime as dt, glob
from openai import OpenAI
from dotenv import load_dotenv; load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# ---- 6A) Build compact aggregates from df (must come BEFORE prompt uses agg_payload) ----
def _norm(s):
    if pd.isna(s): return None
    s = " ".join(str(s).split())
    return s if s else None

df_slim = df.copy()
for col in ["pain","workaround","potential_competitor","verbatim_quote","action","feeling","opportunity"]:
    if col in df_slim.columns:
        df_slim[col] = df_slim[col].apply(_norm)

by_ps = (df_slim.groupby(["persona","stage"], dropna=False)
         .size().reset_index(name="count")
         .sort_values("count", ascending=False))

top_pains = (df_slim["pain"].dropna().str.strip().str.lower()
             .value_counts().head(15).reset_index())
top_pains.columns = ["pain", "count"]

top_workarounds = (df_slim["workaround"].dropna().str.strip()
                   .value_counts().head(15).reset_index())
top_workarounds.columns = ["workaround","count"]

competitors = (df_slim["potential_competitor"].dropna().str.strip()
               .value_counts().head(15).reset_index())
competitors.columns = ["competitor","mentions"]

def sample_rows(n=60):
    pool = df_slim.dropna(subset=["verbatim_quote"])
    if len(pool) <= n: 
        return pool
    out, per_stage = [], max(1, n // max(1, pool["stage"].nunique()))
    for stage, g in pool.groupby("stage"):
        out.append(g.sample(min(per_stage, len(g)), random_state=42))
    return pd.concat(out).head(n)

sample = sample_rows(60)

def to_records_table(frame, cols):
    def trunc(s, n=280):
        s = str(s);  return (s[:n] + "…") if len(s) > n else s
    return [{c: trunc(row.get(c, "")) for c in cols}
            for row in frame[cols].to_dict(orient="records")]

agg_payload = {
    "counts_by_persona_stage": by_ps.to_dict(orient="records"),
    "top_pains": top_pains.to_dict(orient="records"),
    "top_workarounds": top_workarounds.to_dict(orient="records"),
    "competitors": competitors.to_dict(orient="records"),
    "sample_rows": to_records_table(
        sample,
        ["persona","stage","pain","workaround","potential_competitor","verbatim_quote","opportunity","permalink","thread_title"]
    )
}

# ---- 6B) Build scope/method strings (after aggregates, so df exists) ----
today = dt.datetime.utcnow().date()
try:
    RECENCY_MONTHS
except NameError:
    RECENCY_MONTHS = max(1, int(SINCE_DAYS // 30))
start_date = today - dt.timedelta(days=RECENCY_MONTHS * 30)
research_window_str = f"last {RECENCY_MONTHS} months ({start_date.isoformat()} to {today.isoformat()} UTC)"

# prefer bundles_dedup, else count cached files, else len(df)
try:
    thread_count = len(bundles_dedup)
except NameError:
    raw_dir = EXPORTS / f"raw_{RUN_ID}"
    thread_count = len(list((raw_dir).glob("*.json"))) if raw_dir.exists() else None
if not thread_count:
    thread_count = None
sample_counts_str = f"{len(df)} coded rows" + (f" from {thread_count} threads" if thread_count else "") + f" (top {TOP_COMMENTS} comments per thread max)"

sources_str = f"Reddit; subreddits: {', '.join(SUBREDDITS)}"

SYSTEM = (
    "You are a senior UX researcher. Analyze private-party used-car transactions "
    "(buyer and seller dealing directly, not dealers). Find pain points, patterns, and opportunities. "
    "Be concise, evidence-backed, and pragmatic."
)

USER = f"""
You are given aggregated stats and a sample of rows (each with persona, stage, pain, workaround,
competitor mention, and a verbatim quote). Produce a structured MARKDOWN report with:

0) **Scope & Method** — Explicitly state:
   • Research window: {research_window_str}
   • Sources: {sources_str}
   • Sample size: {sample_counts_str}

1) Executive summary (5–8 bullets).
2) Trends by stage (major pain points; which persona; include a short supporting quote ≤120 chars; cite stage).
3) Workarounds & their limits (what users try; risks; where experience breaks).
4) Competitors/solutions mentioned (what they solve vs. gaps; call out named products/services).
5) Opportunities (specific, testable product ideas for concierge/financing/inspection/payment flows).
6) Risks/unknowns (what needs validation in interviews).

Rules:
- Use concise bullet lists. Do NOT invent facts.
- Quotes must be verbatim snippets from provided data (short).
- Prefer patterns supported by multiple rows.

DATA (JSON):
{json.dumps(agg_payload, ensure_ascii=False)}
"""

# ---- 6C) Call the model (Responses API is fine for markdown output) ----
resp = client.responses.create(
    model="gpt-4.1-mini",   # or "o3-mini"
    temperature=0.2,
    input=[
        {"role":"system","content": SYSTEM},
        {"role":"user","content": USER}
    ]
)

insights_md = resp.output_text

# Save and preview
out_md = EXPORTS / f"insights_{RUN_ID}.md"
with open(out_md, "w") as f:
    f.write(insights_md)

print("✅ Insights saved →", out_md)
print("\n--- Preview (first 60 lines) ---")
print("\n".join(insights_md.splitlines()[:60]))


✅ Insights saved → ../data/exports/insights_20251025_1652.md

--- Preview (first 60 lines) ---
# Private-Party Used-Car Transactions: UX Research Report

---

## 0) Scope & Method

- **Research window:** Last 24 months (2023-11-05 to 2025-10-25 UTC)  
- **Sources:** Reddit; subreddits UsedCars, MechanicAdvice, WhatCarShouldIBuy, cars, Scams, AskCarsales  
- **Sample size:** 160 coded rows from 80 threads (max top 4 comments per thread)  

---

## 1) Executive Summary

- **Inspection & condition** is the most frequent pain point for buyers, with distrust of sellers and dealers refusing independent inspections.  
- **Paperwork** complexity and title transfer issues cause significant anxiety and risk, especially with liens and multi-party transactions.  
- **Payment** stage involves high risk of scams, unclear payment guarantees, and challenges coordinating secure fund/title exchange.  
- **Financing private-party sales** is poorly understood, with limited options and lack of clear guidan

In [ ]:
# --- Stage 7: Google Sheets writer (new tab per run) ---

import os, math, datetime as dt, pandas as pd, gspread
from google.oauth2.service_account import Credentials
from pathlib import Path
from dotenv import load_dotenv; load_dotenv()

SHEET_ID = os.getenv("GOOGLE_SHEETS_SPREADSHEET_ID")
assert SHEET_ID, "Missing GOOGLE_SHEETS_SPREADSHEET_ID in .env"

# resolve creds path from env, tolerate running inside notebooks/
raw_key_path = os.getenv("GOOGLE_CREDENTIALS_PATH", "google-credentials.json")
key_path = Path(raw_key_path).expanduser()
if not key_path.exists():
    key_path = Path("..") / raw_key_path
assert key_path.exists(), f"Credentials file not found at '{raw_key_path}' or '{key_path}'"

# auth
creds = Credentials.from_service_account_file(
    str(key_path), scopes=["https://www.googleapis.com/auth/spreadsheets"]
)
gc = gspread.authorize(creds)
ss = gc.open_by_key(SHEET_ID)

# make a deterministic tab name
tab_name = dt.datetime.utcnow().strftime(f"run_{RUN_ID}")

# create worksheet (if name already exists, add a suffix)
try:
    ws = ss.add_worksheet(title=tab_name, rows=2000, cols=30)
except gspread.exceptions.APIError:
    ws = ss.add_worksheet(title=tab_name + "_1", rows=2000, cols=30)

# turn reviewed_rows into a DataFrame (ensures column order)
df = pd.DataFrame(reviewed_rows)

# write headers first
ws.update([df.columns.tolist()])

# write data in chunks (Google API cell size limits)
values = df.astype(object).where(pd.notnull(df), None).values.tolist()
chunk_size = 500  # rows per chunk; tune as needed
for i in range(0, len(values), chunk_size):
    ws.append_rows(values[i:i+chunk_size], value_input_option="RAW")

print(f"✅ Wrote {len(df)} rows to sheet '{ss.title}' tab '{ws.title}'")


OK → User Journey Mapping | using key: ../google-credentials.json
